<a href="https://colab.research.google.com/github/LucianoRodriguez0764/ipadapter_faceid_colab/blob/main/Ipadapter_faceID_Latest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing dependencies and download the IP-Adapter model

In [ ]:
!pip install insightface
!pip install onnxruntime
!pip install diffusers
!pip install git+https://github.com/tencent-ailab/IP-Adapter.git
!pip install einops

# Generate the image with IP-Adapter

In [6]:
!wget -O ip-adapter-faceid-plusv2_sd15.bin https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-plusv2_sd15.bin

--2024-11-20 15:47:50--  https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-plusv2_sd15.bin
Resolving huggingface.co (huggingface.co)... 18.239.50.16, 18.239.50.80, 18.239.50.103, ...
Connecting to huggingface.co (huggingface.co)|18.239.50.16|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/36/ca/36ca54ff1895000df817b45a5ae71fccd14170b0843d8adc2ba7944d9ac903e9/26d0d86a1d60d6cc811d3b8862178b461e1eeb651e6fe2b72ba17aa95411e313?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27ip-adapter-faceid-plusv2_sd15.bin%3B+filename%3D%22ip-adapter-faceid-plusv2_sd15.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1732375712&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczMjM3NTcxMn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzM2L2NhLzM2Y2E1NGZmMTg5NTAwMGRmODE3YjQ1YTVhZTcxZmNjZDE0MTcwYjA4NDNkOGFkYzJiYTc5NDRkOWFjOTAzZTkvMjZk

In [8]:
import torch
from diffusers import StableDiffusionPipeline, DDIMScheduler, AutoencoderKL
from PIL import Image as PILImage

from ip_adapter.ip_adapter_faceid import IPAdapterFaceIDPlus

v2 = True
base_model_path = "SG161222/Realistic_Vision_V4.0_noVAE"
vae_model_path = "stabilityai/sd-vae-ft-mse"
image_encoder_path = "laion/CLIP-ViT-H-14-laion2B-s32B-b79K"
ip_ckpt = "ip-adapter-faceid-plus_sd15.bin" if not v2 else "ip-adapter-faceid-plusv2_sd15.bin"
device = "cuda"

noise_scheduler = DDIMScheduler(
    num_train_timesteps=1000,
    beta_start=0.00085,
    beta_end=0.012,
    beta_schedule="scaled_linear",
    clip_sample=False,
    set_alpha_to_one=False,
    steps_offset=1,
)
vae = AutoencoderKL.from_pretrained(vae_model_path).to(dtype=torch.float16)
pipe = StableDiffusionPipeline.from_pretrained(
    base_model_path,
    torch_dtype=torch.float16,
    scheduler=noise_scheduler,
    vae=vae,
    feature_extractor=None,
    safety_checker=None
)



Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

In [162]:
import cv2
from insightface.app import FaceAnalysis
from insightface.utils import face_align
import torch
from PIL import Image as PILImage

app = FaceAnalysis(name="buffalo_l", providers=['CUDAExecutionProvider', 'CPUExecutionProvider'])
app.prepare(ctx_id=0, det_size=(640, 640))

# face
image1 = cv2.imread("face.jpg")
# structure
image2 = cv2.imread("person.png")
faces1 = app.get(image1)
faces2 = app.get(image2)
if faces1:
  print("person detected")
if faces2:
  print("face detected")

faceid_embeds1 = torch.from_numpy(faces1[0].normed_embedding).unsqueeze(0)
faceid_embeds2 = torch.from_numpy(faces2[0].normed_embedding).unsqueeze(0)
face_image1 = face_align.norm_crop(image1, landmark=faces1[0].kps, image_size=336)
face_image2 = face_align.norm_crop(image2, landmark=faces2[0].kps, image_size=336)


alpha = 0.7  # Weight for person2's identity
blended_embeds2 = alpha * faceid_embeds2 + (1 - alpha) * faceid_embeds1
blended_embeds1 = alpha * faceid_embeds1 + (1 - alpha) * faceid_embeds2

/usr/local/lib/python3.10/dist-packages/onnxruntime/capi/onnxruntime_inference_collection.py:115: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


/usr/local/lib/python3.10/dist-packages/insightface/utils/transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4


person detected
face detected


In [ ]:
face_image2

In [ ]:
# load ip-adapter
ip_ckpt = "ip-adapter-faceid-plusv2_sd15.bin"
ip_model = IPAdapterFaceIDPlus(pipe, image_encoder_path, ip_ckpt, device)

import random

# generate image
prompt = "photo of a 20 year old girl, skinny and bonny"
negative_prompt = "monochrome, lowres, bad anatomy, worst quality, low quality, blurry"
v2=True

seed = random.randint(1,999999)
seed = 44444
print(seed)



images = ip_model.generate(
     prompt=prompt, negative_prompt=negative_prompt, face_image=face_image1, faceid_embeds=blended_embeds2, shortcut=True, s_scale=1,
     num_samples=1, width=512, height=512, num_inference_steps=100, seed=seed, guidance_scale=3
)
images[0]